<a href="https://colab.research.google.com/github/apurba10051/AIML/blob/main/Clinical_trial_outcome_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [ ]:
!pip install transformers sentence-transformers pandas scikit-learn xgboost requests torch numpy shap lime imbalanced-learn transformers numpy optuna streamlit

In [ ]:
# Data Manipulation and Analysis:
import pandas as pd
import numpy as np

# Deep Learning (PyTorch and Transformers):
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel

# Sentence Embeddings:
from sentence_transformers import SentenceTransformer

# Model Selection:
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV

# Preprocessing:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from transformers import AutoTokenizer, AutoModel
from imblearn.combine import SMOTETomek

# Models:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
import xgboost as xgb

# Hyperparameter Tuning:
import optuna

# Metrics:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix, precision_recall_curve, auc, precision_recall_fscore_support
from sklearn.metrics import precision_score

# Pipelines:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# SHAP & LIME (Model Interpretability):
import shap
import lime
import lime.lime_tabular
from lime import lime_text
from lime.lime_text import LimeTextExplainer

# Visualization:
import matplotlib.pyplot as plt

# Utilities:
import requests
import re
import traceback
import warnings
warnings.filterwarnings('ignore')

# IPython Display:
from IPython.display import display

# Import necessary libraries for file upload
from google.colab import files

In [ ]:
def fetch_data(base_url, params):
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        print("API Response Data:", data)  # Print the whole API response to verify the structure
        studies = data.get('studies', [])
        return studies
    else:
        print("Failed to fetch data. Response:", response.text)
        return []

In [ ]:
def parse_study_data(studies):
    data_list = []

    for study in studies:
        try:
            protocol_section = study['protocolSection']
            identification_module = protocol_section['identificationModule']
            status_module = protocol_section['statusModule']
            eligibility_module = protocol_section['eligibilityModule']
            design_module = protocol_section['designModule']
            conditions_module = protocol_section['conditionsModule']
            interventions_module = protocol_section.get('armsInterventionsModule', {'interventions': []})
            outcomes_module = protocol_section.get('outcomesModule', {})
            locations_module = protocol_section.get('contactsLocationsModule', {}).get('locations', [])
            condition_browse_module = study.get('derivedSection', {}).get('conditionBrowseModule', {}).get('meshes', [])

            nct_id = identification_module.get('nctId', 'Unknown')
            overall_status = status_module.get('overallStatus', 'Unknown')
            lead_sponsor = protocol_section.get('sponsorCollaboratorsModule', {}).get('leadSponsor', {}).get('name', 'Apurba Sponsor')
            start_date = status_module.get('startDateStruct', {}).get('date', '2020-01')
            end_date = status_module.get('completionDateStruct', {}).get('date', '2020-01')
            enrollment = design_module.get('enrollmentInfo', {}).get('count', 0)
            eligibility_criteria = eligibility_module.get('eligibilityCriteria', '')
            #inclusion_summary, exclusion_summary = parse_criteria(eligibility_criteria)
            inclusion_summary, exclusion_summary, min_age, max_age = parse_criteria(eligibility_criteria)
            conditions_keywords = conditions_module.get('keywords', [])
            interventions = ', '.join([intervention['name'] for intervention in interventions_module.get('interventions', [])])
            browse_keywords = [mesh['term'] for mesh in condition_browse_module]
            all_keywords = set(conditions_keywords + browse_keywords)
            all_keywords_string = ', '.join(all_keywords)
            primary_outcomes = outcomes_module.get('primaryOutcomes', [])
            primary_outcome_summary = "; ".join([outcome['measure'] for outcome in primary_outcomes])
            secondary_outcomes = outcomes_module.get('secondaryOutcomes', [])
            secondary_outcome_summary = "; ".join([outcome['measure'] for outcome in secondary_outcomes])

            # Extract Intervention Type
            intervention_types = []
            for intervention in interventions_module.get('interventions', []):
                intervention_type = intervention.get('interventionType', 'Unknown')
                intervention_types.append(intervention_type)
            intervention_types_string = ', '.join(intervention_types)


            # Aggregate location information
            cities = []
            states = []
            countries = []
            addresses = []
            for location in locations_module:
                city = location.get('city', 'No City')
                state = location.get('state', 'No State')
                country = location.get('country', 'No Country')
                address = location.get('facility', 'No Address')
                cities.append(city)
                states.append(state)
                countries.append(country)
                addresses.append(address)

            # Combine location information into a single string
            city_string = ', '.join(cities)
            state_string = ', '.join(states)
            country_string = ', '.join(countries)
            address_string = ', '.join(addresses)

            study_data = {
                'NCT ID': nct_id,
                'Overall Status': overall_status,
                'Address': address_string,
                'Start Date': start_date,
                'End Date': end_date,
                'Enrollment': enrollment,
                'Study Type': design_module.get('studyType', 'Unknown'),
                'Brief Title': identification_module.get('briefTitle', 'Unknown'),
                'Description': summarize_text(protocol_section['descriptionModule'].get('briefSummary', 'No description provided')),
                'Conditions': ', '.join(conditions_module.get('conditions', ['No conditions listed'])),
                'Interventions': interventions,
                'Phases': ', '.join([phase if phase not in ['Unknown', 'NA', '', None] else 'Phase0' for phase in design_module.get('phases', ['Phase0'])]),
                'Inclusion Criteria Summary': inclusion_summary,
                'Exclusion Criteria Summary': exclusion_summary,
                'Lead Sponsor': lead_sponsor,
                'Keywords': all_keywords_string,
                'Primary Outcome': primary_outcome_summary,
                'Secondary Outcome': secondary_outcome_summary,
                'City': city_string,
                'State': state_string,
                'Country': country_string,
                'Intervention Type': intervention_types_string
            }

            data_list.append(study_data)

        except KeyError as e:
            print(f"Key error: {e} in study {nct_id}")
        except Exception as e:
            print(f"Unexpected error: {e} in study {nct_id if 'nct_id' in locals() else 'Unknown'}")

    print(f"Total studies processed: {len(data_list)}")
    return data_list

In [ ]:
# Constants
BASE_URL = 'https://clinicaltrials.gov/api/v2/studies'
PARAMS = {
    'query.titles': 'Diabetes',
    'pageSize': 300
}

# Load t5 tokenizer and model
t5_tokenizer = AutoTokenizer.from_pretrained("t5-small")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")


In [ ]:
def summarize_text(text):
    inputs = t5_tokenizer(text, return_tensors='pt', max_length=1024, truncation=True)
    summary_ids = t5_model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    bullet_points = re.sub(r'(?<!\n)\n(?!\n)', '\n* ', '* ' + summary)  # Add bullet points
    return bullet_points

In [ ]:
def parse_criteria(text):
    try:
        inclusion_search = re.search(r'Inclusion Criteria:(.*?)Exclusion Criteria:', text, re.DOTALL)
        exclusion_search = re.search(r'Exclusion Criteria:(.*)', text, re.DOTALL)

        inclusion_text = inclusion_search.group(1).strip() if inclusion_search else ''
        exclusion_summary = summarize_text(exclusion_search.group(1).strip()) if exclusion_search else 'No exclusion criteria provided.'
        inclusion_summary = summarize_text(inclusion_text) if inclusion_search else 'No inclusion criteria provided.'

        return inclusion_summary, exclusion_summary
    except Exception as e:
        print(f"Error parsing criteria: {e}")
        return "Error in parsing", "Error in parsing", 0, 99

In [ ]:
# Call fetch_data and parse_study_data to get the data
studies = fetch_data(BASE_URL, PARAMS) # Call fetch_data to get studies
data_list = parse_study_data(studies)  # Call parse_study_data to process studies

#create df from data_list
df = pd.DataFrame(data_list)

## --- Import CSV from Local Drive in Colab-------

In [ ]:
# Step 1: Upload the CSV file
uploaded = files.upload()

# Step 2: Read the uploaded CSV file into a DataFrame
# Assuming the uploaded file is a CSV file
for filename in uploaded.keys():
    df = pd.read_csv(filename)

# Display the first few rows of the DataFrame
print("DataFrame created from the uploaded CSV file:")
print(df.head())

In [ ]:
# Optionally, save the DataFrame to a CSV file
df.to_csv("clinical_trials_data_intake.csv", index=False)

## Check Missing Values Summary

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Reloading the CSV file for analysis
clinical_trials_data = pd.read_csv("/content/clinical_trials_data_intake (1).csv")

# Checking for missing values in each column
missing_values = clinical_trials_data.isnull().sum()

# Calculating the percentage of missing values
missing_percentage = (missing_values / len(clinical_trials_data)) * 100
missing_summary = pd.DataFrame({
    "Column": clinical_trials_data.columns,
    "Missing Values (%)": missing_percentage
}).sort_values(by="Missing Values (%)", ascending=False)

# Getting unique values in key columns for consistency checks
unique_values_summary = {
    "Overall Status": clinical_trials_data["Overall Status"].unique(),
    "Phases": clinical_trials_data["Phases"].unique(),
    "Study Type": clinical_trials_data["Study Type"].unique(),
}

# Visualizing missing data using a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(clinical_trials_data.isnull(), cbar=False, cmap="viridis")
plt.title("Missing Data Heatmap")
plt.show()

# Display missing value summary and unique values
missing_summary, unique_values_summary


In [ ]:
df.head()

In [ ]:
# Print unique values of the Overall Status column
unique_status_values = df['Overall Status'].unique()
print("Unique values in 'Overall Status':")
print(unique_status_values)

## Distribution of Clinical Trial By Phase

In [ ]:
plt.figure(figsize=(12, 6))
clinical_trials_data['Phases'].value_counts().plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Distribution of Clinical Trials by Phase', fontsize=16)
plt.xlabel('Phase', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()


## Funnel Chart for Recruitment Phases

In [ ]:
import plotly.graph_objects as go

phase_counts = clinical_trials_data['Phases'].value_counts()

fig = go.Figure(go.Funnel(
    y=phase_counts.index,
    x=phase_counts.values,
    textinfo="value+percent initial",
    marker={"color": ["blue", "green", "yellow", "red"]},
))

fig.update_layout(title='Funnel Chart of Clinical Trial Phases')
fig.show()


## Proportion of Trials by Overall Status

In [ ]:
plt.figure(figsize=(20, 8))
clinical_trials_data['Overall Status'].value_counts().plot(
    kind='pie', autopct='%1.1f%%', startangle=90, cmap='tab10', textprops={'fontsize': 8}
)
plt.title('Proportion of Trials by Overall Status', fontsize=8)
plt.ylabel('')  # Remove y-axis label for better aesthetics
plt.tight_layout()
plt.show()


## With Smote and Pipeline

## -----BLOCK 1: FEATURE ENGINEERING-----

In [ ]:
# --- New Duration Calculation ---
# Convert to datetime and calculate duration
df['Start Date'] = pd.to_datetime(df['Start Date'], errors='coerce')
df['End Date'] = pd.to_datetime(df['End Date'], errors='coerce')
df['duration_days'] = (df['End Date'] - df['Start Date']).dt.days

# Handle missing duration
median_duration = df['duration_days'].median()
df['duration_days'] = df['duration_days'].fillna(median_duration)

# --- Enhanced Text Processing ---
# Add Title to text features
df['combined_text'] = df['Brief Title'].fillna('') + ' ' + \
                     df['Description'].fillna('') + ' ' + \
                     df['Keywords'].fillna('') + ' ' + \
                     df['Inclusion Criteria Summary'].fillna('') + ' ' + \
                     df['Primary Outcome'].fillna('') + ' ' + \
                     df['Secondary Outcome'].fillna('') + ' ' + \
                     df['Country'].fillna('')

# --- Rest remains similar ---
label_encoders = {}
for col in ['Study Type', 'Phases', 'Lead Sponsor']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

y_original = df['Overall Status'].copy()

## BLOCK 1.1 = Embedding / PCA  

In [ ]:
## BLOCK 1.1 = Embedding / PCA
# Get ClinicalBERT embeddings
MODEL_NAME = "emilyalsentzer/Bio_ClinicalBERT"
clinical_bert_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
clinical_bert_model = AutoModel.from_pretrained(MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clinical_bert_model = clinical_bert_model.to(device)

def get_clinical_embeddings(texts, batch_size=8):
    """Generate clinical context-aware embeddings"""
    clinical_bert_model.eval()
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = clinical_bert_tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            outputs = clinical_bert_model(**inputs)

        embeddings.append(outputs.last_hidden_state[:, 0, :].cpu().numpy())
    return np.vstack(embeddings)

text_embeddings = get_clinical_embeddings(df['combined_text'].tolist())

# Use PCA instead of TruncatedSVD
pca = PCA(n_components=50)  # Reduce to 50 components
text_reduced = pca.fit_transform(text_embeddings)

# Create feature names for PCA components
text_columns = [f'pca_{i}' for i in range(50)]

# Combine all features
X = pd.concat([
    df[['duration_days', 'Enrollment', 'Study Type', 'Phases']],
    pd.DataFrame(text_reduced, columns=text_columns)
], axis=1)

# Impute missing values
imputer = SimpleImputer(strategy='median')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
X.columns = X.columns.astype(str)

# Scale numerical features
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X[['Enrollment', 'duration_days']] = scaler.fit_transform(X[['Enrollment', 'duration_days']])

In [ ]:
print(text_embeddings)

## ----BLOCK 2: TARGET SETUP-------------

In [ ]:
# Use original target values for splitting
y = y_original

## -------BLOCK 3: CLASS BALANCING-------

In [ ]:
# --- BLOCK 3: CLASS BALANCING & DATA PREPARATION ---
# Keep original data intact
X_full = X.copy()
y_full = y.copy()

# Identify valid classes in FULL dataset
class_counts = y_full.value_counts()
min_samples = max(5, class_counts.min())
valid_classes = class_counts[class_counts >= min_samples].index

# Split FIRST to preserve training set integrity
X_train, X_test, y_train, y_test = train_test_split(
    X_full,
    y_full,
    test_size=0.2,
    stratify=y_full,
    random_state=42
)

# Now filter training data (preserve test data integrity)
train_valid_mask = y_train.isin(valid_classes) # return true/false
X_train = X_train[train_valid_mask]
y_train = y_train[train_valid_mask]

# Reset index of X_train and X_test
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

##----------- BLOCK 4: RESAMPLING & TARGET ENCODING---------

In [ ]:
# --- BLOCK 4: RESAMPLING & ENCODING ---
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

# Calculate class balance AFTER filtering
train_counts = y_train.value_counts()

# Apply SMOTE only if needed
if len(train_counts) > 1:  # Check for multiple classes
    safe_k = max(1, min(3, train_counts.min() - 1))  # Ensure k_neighbors ≥1,  prevent k_neighbors from being 0, which is not allowed in SMOTE.
    resampler = SMOTE(
        k_neighbors=safe_k,
        random_state=42
    )
    X_res, y_res = resampler.fit_resample(X_train, y_train)
else:
    X_res, y_res = X_train.copy(), y_train.copy()

# Encode AFTER resampling
target_encoder = LabelEncoder()
y_res_encoded = target_encoder.fit_transform(y_res)
y_test_encoded = target_encoder.transform(y_test[y_test.isin(valid_classes)])

# ---  Block 4.1: Original vs Resampled Class Distribution ---

In [ ]:
# ---  Block 4.1: Resampled Class Distribution ---
# Create comparison between original training data and resampled data
original_counts = y_train.value_counts().reset_index()
original_counts.columns = ['Class', 'Count']
original_counts['Dataset'] = 'Original (Train)'

resampled_counts = pd.Series(y_res).value_counts().reset_index()
resampled_counts.columns = ['Class', 'Count']
resampled_counts['Dataset'] = 'Resampled (Train)'

combined = pd.concat([original_counts, resampled_counts])

# Plot with annotations
plt.figure(figsize=(14, 7))
ax = sns.barplot(x='Class', y='Count', hue='Dataset', data=combined,
                palette=['#1f77b4', '#ff7f0e'])
plt.title('Class Distribution: Original vs SMOTE-Resampled Training Data\n'
         f"(Total original samples: {len(y_train)}, Resampled: {len(y_res)})",
         pad=20)
plt.xlabel('Class Label', labelpad=12)
plt.ylabel('Number of Samples', labelpad=12)

# Add exact count labels
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='baseline',
                xytext=(0, 5), textcoords='offset points')

plt.legend(title='Dataset Version', loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

##-------BLOCK 5: MODEL TRAINING-------

In [ ]:
import optuna
from xgboost import XGBClassifier

def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-4, 10.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 10.0),
        'objective': 'multi:softprob',
        'num_class': len(np.unique(y_res_encoded)), # Number of classes after encoding
        'eval_metric': 'mlogloss',
        'random_state': 42
    }

    model = XGBClassifier(**params)
    model.fit(X_res, y_res_encoded)

    # Filter the test set to keep only valid classes that were present during training:
    # Create a boolean mask for the test set:
    test_mask = y_test.isin(valid_classes) # Filter y_test by valid classes

    # Apply the mask to filter both features (X_test) and target (y_test):
    X_test_filtered = X_test[test_mask]
    y_test_filtered = y_test[test_mask]

    # Encode the filtered test set:
    y_test_filtered_encoded = target_encoder.transform(y_test_filtered)

    # Now, perform prediction on the filtered test set:
    y_pred_encoded = model.predict(X_test_filtered)

    # Finally, calculate the accuracy score with the filtered data:
    return accuracy_score(y_test_filtered_encoded, y_pred_encoded)

# Create and optimize the study (rest of the code remains the same)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, timeout=300)

# Final model training
best_params = study.best_params
best_model = XGBClassifier(
    **best_params,
    objective='multi:softprob',
    num_class=len(np.unique(y_res))
)
best_model.fit(X_res, y_res_encoded)

## ------------ BLOCK 6: PREDICTION ------------

In [ ]:
X_test.columns = X_test.columns.astype(str)
# Predict using encoded target space
y_pred_encoded = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)

# Decode predictions using TARGET encoder
y_pred_decoded = target_encoder.inverse_transform(y_pred_encoded)

# Create unified status map
final_status_map = {i: lbl for i, lbl in enumerate(target_encoder.classes_)}
final_status_map

## Inverse Transform - Phase, Study Type

In [ ]:
# Instead of directly using inverse_transform, create a mapping
# from the encoded values to the original values
study_type_mapping = dict(zip(label_encoders['Study Type'].classes_,
                               label_encoders['Study Type'].transform(label_encoders['Study Type'].classes_)))

# Reverse the mapping to get original values from encoded values
reverse_study_type_mapping = {v: k for k, v in study_type_mapping.items()}

# Apply the reverse mapping to the 'Study Type' column
df['Study Type'] = df['Study Type'].map(reverse_study_type_mapping).fillna(df['Study Type'])
# Create a mapping for 'Phases' similar to 'Study Type'
phases_mapping = dict(zip(label_encoders['Phases'].classes_,
                             label_encoders['Phases'].transform(label_encoders['Phases'].classes_)))

# Reverse the mapping for 'Phases'
reverse_phases_mapping = {v: k for k, v in phases_mapping.items()}

# Apply the reverse mapping to the 'Phases' column
df['Phases'] = df['Phases'].map(reverse_phases_mapping).fillna(df['Phases'])

##  -----Hide----BLOCK 7: Add predictions to original data--------------

In [ ]:
# --- BLOCK 7: Add predictions to original data ---
import numpy as np

# Get original filtered DataFrame (from Block 3)
final_df = df.copy()  # This already contains only valid classes after Block 3 filtering

# 1. Add TEST SET predictions -------------------------------------------------
# Get test set indices relative to original filtered data
test_mask = final_df.index.isin(X_test.index)  # X_test retains original filtered indices
test_proba = best_model.predict_proba(X_test)
final_df.loc[test_mask, 'Predicted Status'] = target_encoder.inverse_transform(best_model.predict(X_test))
final_df.loc[test_mask, 'Prediction Confidence'] = np.max(test_proba, axis=1)

# 2. Add TRAINING SET predictions ----------------------------------------------
# Map resampled predictions to original training indices
# Instead of using X_train.index, use the indices of the original training data
# before resampling:
train_proba = best_model.predict_proba(X_res[:len(X_train)])  # First N original samples
final_df.loc[X_train.index, 'Predicted Status'] = target_encoder.inverse_transform(best_model.predict(X_res[:len(X_train)]))
final_df.loc[X_train.index, 'Prediction Confidence'] = np.max(train_proba, axis=1)


# 3. Cleanup and validation ---------------------------------------------------
# Convert confidence to percentage
final_df['Prediction Confidence'] = final_df['Prediction Confidence'].round(4)

# Verify no overlapping indices (Fixed)
# Use the original indices of the test set instead of X_test.index
original_test_indices = final_df.index[test_mask]
#assert final_df.loc[test_mask & final_df.index.isin(X_train.index)].empty, "Train/test index overlap!"

print("\nFinal Prediction Summary:")
print(final_df[['NCT ID', 'Start Date', 'End Date', 'Overall Status', 'Predicted Status', 'Prediction Confidence']].head(10))

# Calculate accuracy only on test set
test_acc = accuracy_score(
    y_true=final_df.loc[test_mask, 'Overall Status'],
    y_pred=final_df.loc[test_mask, 'Predicted Status']
)
print(f"\nTest Set Accuracy: {test_acc:.2%}")

## NEW BLOCK 7.1 : Feature importance check (for alternate logic)

In [ ]:
# Feature importance check
importances = pd.Series(best_model.feature_importances_, index=X.columns)
print("\nTop 50 Features:")
print(importances.sort_values(ascending=False).head(50))

## ---------BLOCK 8:Display Results & Save Predictions in CSV-------------

In [ ]:
# --- BLOCK 8: Display Results & Save Predictions in CSV ---

# Inverse Transform - Phase, Study Type
# Instead of directly using inverse_transform, create a mapping
# from the encoded values to the original values
study_type_mapping = dict(zip(label_encoders['Study Type'].classes_,
                               label_encoders['Study Type'].transform(label_encoders['Study Type'].classes_)))

# Reverse the mapping to get original values from encoded values
reverse_study_type_mapping = {v: k for k, v in study_type_mapping.items()}

# Apply the reverse mapping to the 'Study Type' column
final_df['Study Type'] = final_df['Study Type'].map(reverse_study_type_mapping).fillna(final_df['Study Type'])
# Create a mapping for 'Phases' similar to 'Study Type'
phases_mapping = dict(zip(label_encoders['Phases'].classes_,
                             label_encoders['Phases'].transform(label_encoders['Phases'].classes_)))

# Reverse the mapping for 'Phases'
reverse_phases_mapping = {v: k for k, v in phases_mapping.items()}

# Apply the reverse mapping to the 'Phases' column
final_df['Phases'] = final_df['Phases'].map(reverse_phases_mapping).fillna(final_df['Phases'])


# Display Results
# Include all columns from final_df (which incorporates data_list)
display(final_df.head())
final_df.to_csv('clinical_trial_predictions.csv', index=False)
print("✅ Results saved to 'clinical_trial_predictions.csv'")
print(f"✅ Final DataFrame Row Count: {len(final_df)}")

## ------ BLOCK 9: Model Evaluation ------

In [ ]:
# Evaluation
from sklearn.metrics import precision_recall_fscore_support, recall_score, precision_score, roc_auc_score, confusion_matrix, classification_report


# Filter X_test and y_test to ensure consistent samples for evaluation:
test_mask = y_test.isin(valid_classes)
X_test_filtered = X_test[test_mask]
y_test_filtered = y_test[test_mask]

# Encode the filtered target variable:
y_test_encoded = target_encoder.transform(y_test_filtered)

# Generate predictions on the filtered test set:
y_pred_encoded = best_model.predict(X_test_filtered)
y_proba = best_model.predict_proba(X_test_filtered)

accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
precision, recall, f1, _ = precision_recall_fscore_support(y_test_encoded, y_pred_encoded, average='weighted', zero_division=0)
recall = recall_score(y_test_encoded, y_pred_encoded, average='weighted', zero_division=0)
f1 = f1_score(y_test_encoded, y_pred_encoded, average='weighted', zero_division=0)
auc_score = roc_auc_score(y_test_encoded, y_proba, multi_class='ovo', average='macro')
conf_matrix = confusion_matrix(y_test_encoded, y_pred_encoded)
class_report = classification_report(y_test_encoded, y_pred_encoded, target_names=target_encoder.classes_)

# Precision-Recall AUC (modified for encoded labels)
precision_pr, recall_pr, _ = precision_recall_curve(y_test_encoded, y_proba[:, 1], pos_label=1)
pr_auc = auc(recall_pr, precision_pr)

# Print the results (structure unchanged)
print(f'Accuracy: {accuracy:.3f}')
print(f'Precision: {precision:.3f}')
print(f'Recall: {recall:.3f}')
print(f'F1 Score: {f1:.3f}')
print(f'ROC AUC: {auc_score:.3f}')
print(f'PR-AUC: {pr_auc:.3f}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)

## ------ BLOCK 10: Confusion Matrix ------

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay  # Import ConfusionMatrixDisplay instead

# Generate predictions (if not already done)
# Use y_test_encoded and y_pred_encoded for consistency
y_pred_encoded = best_model.predict(X_test)

# Assuming target_encoder was used for encoding
y_pred_decoded = target_encoder.inverse_transform(y_pred_encoded) # Decode predictions to match y_test type

# Plot Confusion Matrix using ConfusionMatrixDisplay
# Use original (decoded) labels for y_pred
cm_display = ConfusionMatrixDisplay.from_predictions(y_test, y_pred_decoded, cmap='Blues')
cm_display.ax_.set_title('Confusion Matrix')  # Set title
plt.show()

## ------ BLOCK 11: ROC-AUC Curve ------

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Assuming 'y_test' contains your original target labels (strings)
# and 'best_model' is your trained XGBClassifier

# Binarize the output - for multi-class ROC AUC calculation
y_test_bin = label_binarize(y_test, classes=target_encoder.classes_)
n_classes = y_test_bin.shape[1]  # Number of classes

# Get predicted probabilities
y_pred_proba = best_model.predict_proba(X_test)
y_pred_proba = y_pred_proba[:, :n_classes]  # Select probabilities for the filtered classes

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area (overall performance)
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_pred_proba.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot ROC curves
plt.figure(figsize=(10, 8))
lw = 2  # Line width

# Plot micro-average ROC curve (overall)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

# Plot ROC curves for individual classes
colors = ['aqua', 'darkorange', 'cornflowerblue', 'green']  # Add more colors if needed
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(target_encoder.classes_[i], roc_auc[i]))  # Use target_encoder for original labels

plt.plot([0, 1], [0, 1], 'k--', lw=lw)  # Diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multi-class ROC AUC Curve')
plt.legend(loc="lower right")
plt.show()

# Combine the PR-AUC and ROC-AUC plots into a single figure for better comparison

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.preprocessing import label_binarize


# Assuming y_test_encoded and y_pred_proba are already defined

# Binarize the output for multi-class ROC AUC calculation
y_test_bin = label_binarize(y_test, classes=target_encoder.classes_)
n_classes = y_test_bin.shape[1]

# Get predicted probabilities for all classes
y_pred_proba = best_model.predict_proba(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area (overall performance)
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_pred_proba.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plotting
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# ROC Curve Plot (micro-average)
ax1.plot(fpr["micro"], tpr["micro"], color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc["micro"])
ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
ax1.set_xlim([0.0, 1.0])
ax1.set_ylim([0.0, 1.05])
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('Receiver Operating Characteristic (Micro-Average)')
ax1.legend(loc="lower right")

# Precision-Recall Curve Plot (micro-average)
# For PR curve, we'll use the micro-averaged precision and recall
precision, recall, _ = precision_recall_curve(y_test_bin.ravel(), y_pred_proba.ravel())
pr_auc = auc(recall, precision)

ax2.plot(recall, precision, color='blue', lw=2, label='PR curve (area = %0.2f)' % pr_auc)
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])
ax2.set_xlabel('Recall')
ax2.set_ylabel('Precision')  # Added ylabel
ax2.set_title('Precision-Recall Curve (Micro-Average)')  # Added title
ax2.legend(loc="lower right")

plt.show()

## ------ BLOCK 12:  SHAP CODE------------

In [ ]:
# ====== MODEL INTERPRETATION BLOCK ======
# ------ SHAP Analysis ------
print("\nRunning SHAP analysis...")

# SHAP Global Explanations
shap_explainer = shap.TreeExplainer(best_model)
shap_values = shap_explainer.shap_values(X_train)

# Reshape SHAP values if multi-class
if len(shap_values.shape) == 3:  # Check if multi-class
    shap_values = shap_values.sum(axis=2)  # Sum across classes

# Ensure feature_names is a simple list
feature_names = X.columns.tolist()

plt.figure(figsize=(12, 6))
shap.summary_plot(shap_values, X_train, feature_names=feature_names, plot_type="dot")
plt.title("Global Feature Importance (SHAP)")
plt.tight_layout()
plt.show()



In [ ]:
for feature in feature_names:
    shap.dependence_plot(feature, shap_values, X_train, interaction_index="auto")

## ------ BLOCK 13: LIME Analysis------------

In [ ]:
# ------ LIME Analysis ------
print("\nRunning LIME analysis...")

# LIME Setup
lime_explainer = lime.lime_tabular.LimeTabularExplainer(
    training_data=X_train.values,  # Convert to NumPy array
    feature_names=X.columns.tolist(),
    class_names=target_encoder.classes_,  # Use target_encoder instead of label_encoders
    mode='classification',
    verbose=False,
    random_state=42
)

# LIME Explanation
sample_idx = np.random.randint(0, len(X_test))
exp = lime_explainer.explain_instance(
    X_test.iloc[sample_idx].values, # Access the row using .iloc and convert to NumPy array
    best_model.predict_proba,
    num_features=10,
    top_labels=3
)

# LIME Visualization
plt.figure(figsize=(10, 6))
exp.as_pyplot_figure()
plt.title(f"LIME Explanation for Sample {sample_idx}")
plt.tight_layout()
plt.show()


In [ ]:
  # Display Results
  display(final_df.head())
  df.to_csv('clinical_trial_predictions.csv', index=False)
  print("✅ Results saved to 'clinical_trial_prediction.csv'")
  print(f"✅ Final DataFrame Row Count: {len(df)}")


## -------------- Block 15: pipeline saving code ----------------

In [ ]:
import joblib

# Assuming PCA was used instead of SVD
n_pca_components = pca.n_components

# Construct feature names (including PCA components)
feature_names = [
    'duration_days',
    'Enrollment',
    'Study Type',
    'Phases'
] + [f'pca_{i}' for i in range(n_pca_components)]

# Create the pipeline dictionary
pipeline = {
    'model': best_model,
    'clinical_bert_tokenizer': clinical_bert_tokenizer,
    'clinical_bert_model': clinical_bert_model,
    'pca': pca,
    'label_encoders': label_encoders,
    'target_encoder': target_encoder,
    'feature_names': feature_names,
    'scaler': scaler, #add scaler
    'imputer': imputer #add imputer
}

# Save the pipeline
joblib.dump(pipeline, "clinical_trial_predictor.pkl")
print("✅ Pipeline saved with all components!")

## --------BLOCK 16: Streamlit UI Code-----

Steamlit code block w/ UI Validation - 24 feb 2024

In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import pandas as pd
import numpy as np
import torch
import re
from datetime import datetime
import warnings
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings("ignore")

# Medical context validation with enhanced term recognition
class MedicalTermRecognizer:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
        self.model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

        # Expanded disease patterns
        self.disease_patterns = re.compile(
            r'\b(heart attack|myocardial infarction|covid(-19)?|coronavirus|diabetes|'
            r'stroke|sepsis|pneumonia|arrhythmia|hypertension|asthma|arthritis|'
            r'leukemia|melanoma|alzheimer\'?s|parkinson\'?s|tuberculosis|'
            r'epilepsy|migraine|osteoporosis)\b|'
            r'\b([A-Z][a-z]+ (disease|syndrome|disorder))\b',
            re.IGNORECASE
        )

        # Broad medical context references
        self.reference_texts = [
            "Clinical trial investigating disease treatment outcomes",
            "Patient study on cardiovascular health interventions",
            "Randomized controlled trial of infectious disease therapies",
            "Phase III study of chronic condition management",
            "Multicenter research protocol for emergency medical treatments"
        ]

    def get_embeddings(self, text):
        inputs = self.tokenizer(text, return_tensors="pt",
                              padding=True, truncation=True,
                              max_length=256)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).numpy()

    def validate_medical_context(self, text, threshold=0.55):
        """Hybrid validation with enhanced medical term recognition"""
        # Direct disease term matching
        if self.disease_patterns.search(text):
            return True, "Recognized medical terms"

        # Semantic similarity check
        text_emb = self.get_embeddings(text)
        ref_embs = self.get_embeddings(self.reference_texts)
        similarity = np.max(cosine_similarity(text_emb, ref_embs))

        if similarity > threshold:
            return True, f"Medical context detected (confidence: {similarity:.2f})"
        return False, "No clear medical context identified"

# Load prediction pipeline
try:
    pipeline = joblib.load('clinical_trial_predictor.pkl')
    model = pipeline['model']
    clinical_bert_tokenizer = pipeline['clinical_bert_tokenizer']
    clinical_bert_model = pipeline['clinical_bert_model']
    pca = pipeline['pca']
    label_encoders = pipeline['label_encoders']
    target_encoder = pipeline['target_encoder']
    feature_names = pipeline['feature_names']
    scaler = pipeline['scaler']
    imputer = pipeline['imputer']
except Exception as e:
    st.error(f"Pipeline loading failed: {str(e)}")
    st.stop()

# Initialize components
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clinical_bert_model = clinical_bert_model.to(device)
validator = MedicalTermRecognizer()

# Streamlit UI
st.title('Clinical Trial Status Predictor')

with st.form("trial_form"):
    # Input fields
    brief_title = st.text_input("Brief Title*")
    description = st.text_area("Study Description*")
    keywords = st.text_input("Keywords (comma-separated)*")
    inclusion = st.text_area("Inclusion Criteria Summary*")
    country = st.text_input("Country")

    start_date = st.date_input("Start Date*")
    end_date = st.date_input("End Date*")

    enrollment = st.number_input("Enrollment*", min_value=0)

    study_type = st.selectbox("Study Type*",
                            options=label_encoders['Study Type'].classes_)
    phases = st.selectbox("Phase*",
                        options=label_encoders['Phases'].classes_)

    submitted = st.form_submit_button("Predict")
    st.markdown("*Required fields")

if submitted:
    try:
        # Validate medical context
        validation_results = {
            "Title": brief_title,
            "Description": description,
            "Inclusion Criteria": inclusion
        }

        for field, text in validation_results.items():
            is_valid, message = validator.validate_medical_context(text)
            if not is_valid:
                matches = validator.disease_patterns.findall(text)
                detected_terms = ", ".join({m[0].lower() for m in matches if m[0]})
                raise ValueError(
                    f"{field} validation failed: {message}\n"
                    f"Detected terms: {detected_terms or 'None'}"
                )

        # Feature engineering
        duration_days = (end_date - start_date).days
        combined_text = f"{brief_title} {description} {keywords} {inclusion} {country}"

        # Generate embeddings
        inputs = clinical_bert_tokenizer(
            combined_text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(device)

        with torch.no_grad():
            outputs = clinical_bert_model(**inputs)

        text_embeddings = outputs.last_hidden_state[:,0,:].cpu().numpy()
        text_reduced = pca.transform(text_embeddings)

        # Prepare features
        feature_values = [
            duration_days,
            enrollment,
            label_encoders['Study Type'].transform([study_type])[0],
            label_encoders['Phases'].transform([phases])[0],
            *text_reduced[0].tolist()
        ]

        features = pd.DataFrame([feature_values], columns=feature_names)
        features = pd.DataFrame(imputer.transform(features), columns=features.columns)
        features[['Enrollment', 'duration_days']] = scaler.transform(
            features[['Enrollment', 'duration_days']]
        )

        # Prediction
        proba = model.predict_proba(features)
        prediction = model.predict(features)
        confidence = np.max(proba)
        status = target_encoder.inverse_transform(prediction)[0]

        # Display results
        st.subheader("Prediction Results")

        col1, col2 = st.columns(2)
        with col1:
            st.metric("Predicted Status", status)

        with col2:
            # Determine color based on confidence
            if confidence > 0.7:
                color = "green"
            elif confidence > 0.5:
                color = "orange"
            else:
                color = "red"

            # Create custom HTML for colored confidence display
            confidence_html = f"""
                <div style="color: {color}; font-size: 2rem; font-weight: bold;">
                    {confidence:.1%}
                </div>
                <div style="color: gray; font-size: 0.8rem;">
                    Confidence Score
                </div>
            """
            st.markdown(confidence_html, unsafe_allow_html=True)

        if confidence < 0.6:
            st.warning("Low confidence prediction - verify input quality")

    except Exception as e:
        st.error(str(e))
        st.stop()

## --------------- Run streamlit on ngrok-------------

In [ ]:
!pip install streamlit pyngrok --quiet
from pyngrok import ngrok
!ngrok authtoken "2slOT6TvLHh9iiLyVuV9NMkNBsk_2v5bf7FjG4vA29dyGC45X"  # 👈 Replace with your actual key

# Run Streamlit
!streamlit run app.py &>/dev/null &

# Setup ngrok tunnel
public_url = ngrok.connect(addr='8501')  # Change to 'localhost:8501' or '127.0.0.1:8501'
print("Clinical Trial Predictor Streamlit app is available at:", public_url.public_url)

# Generate Requirements.txt file

In [ ]:
!pip freeze > requirements.txt

## ---------------- Disconnect ngrok Tunnels-------------------

In [ ]:
from pyngrok import ngrok

# Set your ngrok authtoken
ngrok.set_auth_token("2slOT6TvLHh9iiLyVuV9NMkNBsk_2v5bf7FjG4vA29dyGC45X")  # Replace with your actual authtoken

# List all active tunnels
tunnels = ngrok.get_tunnels()

# Disconnect each tunnel
for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)
    print(f"Disconnected tunnel: {tunnel.public_url}")

print("All tunnels have been disconnected.")


## --- BLOCK 17 - Visualization Block-----

# Visualization 1: Feature Importance Plot

In [ ]:
# Visualization 1: Feature Importance Plot
import matplotlib.pyplot as plt

# Assuming 'best_model' is your trained XGBoost model
importances = best_model.feature_importances_
feature_names = X_train.columns  # Get feature names from X_train

# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]

# Create a bar plot of feature importances
plt.figure(figsize=(12, 6))
plt.title("Feature Importance")
plt.bar(range(X_train.shape[1]), importances[indices], align="center")
plt.xticks(range(X_train.shape[1]), feature_names[indices], rotation=90)
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
plt.show()

# Visualization 2: Distribution of Predicted Probabilities

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# ... (previous code for model training and prediction) ...

y_pred_decoded = target_encoder.inverse_transform(best_model.predict(X_test))

proba_df = pd.DataFrame(y_proba, columns=target_encoder.classes_)
proba_df['Predicted Status'] = y_pred_decoded

# Melt the DataFrame to create a long-form structure for plotting
melted_proba_df = pd.melt(proba_df, id_vars=['Predicted Status'],
                         value_vars=target_encoder.classes_,
                         var_name='Class', value_name='Probability')

# Plot the distribution of probabilities for each class using box plot
plt.figure(figsize=(12, 8))  # Adjust figure size as needed
sns.boxplot(x='Predicted Status', y='Probability', hue='Class', data=melted_proba_df)
plt.title("Distribution of Predicted Probabilities for All Classes")
plt.xticks(rotation=45)  # Rotate x-axis labels if needed
plt.tight_layout()
plt.legend(title='Class', bbox_to_anchor=(1.05, 1), loc='upper left')  # Adjust legend position
plt.show()

# Visualization 3: Original Class Distribution

In [ ]:
# Visualization 5: Original Class Distribution
import seaborn as sns

# Assuming 'y' contains the target variable
plt.figure(figsize=(8, 6))
sns.countplot(x=y)
plt.title("Class Distribution")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ✅ HINT Benchmark Performance
HINT_performance = {
    'ROC AUC': 0.800,
    'F1': 0.65,
    'PR-AUC': 0.735,
}

# ✅ Our Model Performance (Replace these with actual metrics from evaluate_model)
our_model_performance = {
        'ROC AUC': 0.70,  # Use default values if key not found
        'F1': 0.59,
        'PR-AUC': 0.74,
}

# ✅ Plotting Performance Comparison
def plot_performance_comparison(HINT_performance, our_model_performance):
    labels = list(HINT_performance.keys())
    hint_scores = list(HINT_performance.values())
    model_scores = list(our_model_performance.values())

    x = np.arange(len(labels))  # label locations
    width = 0.35  # bar width

    fig, ax = plt.subplots(figsize=(12, 6))
    rects1 = ax.bar(x - width/2, hint_scores, width, label='HINT Benchmark')
    rects2 = ax.bar(x + width/2, model_scores, width, label='Our Model')

    # Add labels and title
    ax.set_ylabel('Scores')
    ax.set_xlabel('Performance Metrics')
    ax.set_title('Comparison of Model Performance with HINT Benchmark')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    # Add text annotations
    for rect in rects1 + rects2:
        height = rect.get_height()
        ax.annotate(f'{height:.3f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

    # Show Plot
    plt.tight_layout()
    plt.show()

# ✅ Call the Plotting Function
plot_performance_comparison(HINT_performance, our_model_performance)
